In [ ]:
!pip install openai numpy

In [ ]:
import requests
import openai
import numpy as np
import numpy.typing as npt

BASE_URL = "http://127.0.0.1:8000"
API_KEY = "n/a"
MODEL = 'mistralai/Mistral-7B-Instruct-v0.2'

In [ ]:
# Exercise 1
#
# Find the correct set of arguments which will ensure the model always
# outputs 'Hello' regardless of temperature. Consider how the sampling
# process occurs on the backend and what you can control.
#
# hint: Check the available options in vllm for our CompletionRequest model

client = openai.OpenAI(api_key=API_KEY, base_url=f"{BASE_URL}/v1")

completion = client.completions.create(
    model=MODEL,
    prompt="<s>[INST] Say hello! [/INST]",
    temperature=0, # Change this and see what happens!
    max_tokens=1,
)

print(completion.choices[0].text)

# Extra credit: Can you get the log probability of the token "Hi"?

In [ ]:
# Exercise 2
#
# Author a function which will return the top-k log probabilities for a given
# prompt using the openai completions endpoint. This should be dict with the
# keys being the token string and the values being the log probability
#
# hint: Check out the logprobs option and the return structure

def topk(prompt: str) -> dict[str, float]:
    # Your code here
    return {}

probs = topk("<s>[INST] Say hello! [/INST]")
print(probs)

assert isinstance(probs, dict) and 'Hello' in probs, "Uh oh!"

In [ ]:
# Exercise 4
#
# Extend/rewrite your topk function to take an optional logit_bias argument.
# Use this new function to force the model to reply "No" when we ask it to
# say "Hello".
#
# hint: Check out the logit_bias_logits_processor in vllm
# hint: Check out the tokenizer.json for the Mistral model 

def topk_with_bias(
    prompt: str, logit_bias: dict[int, float] | None = None
) -> dict[str, float]:
    # Complete this code
    return {}

prompt = "<s>[INST] Say hello! [/INST]"
probs = topk_with_bias(prompt, {1510: 100})
print(probs)

assert next(iter(probs.keys())).lower() == "no", "Uh oh!"

In [ ]:
# Exercise 4
#
# As we bring things together, we're going to be getting the true logprob for our
# 'no' token and ensure that it roughly matches the output from the server if we
# could get all the logprobs as opposed to just the top 5.
#
# References:
# - https://github.com/justinchiu/openlogprobs
# - https://mattf1n.github.io/openlogprobs.html
#

prompt = "<s>[INST] Say hello! [/INST]"
target_token_id = 1510
target_token = "no"
bias = 5.0

original = topk_with_bias(prompt)

modified = topk_with_bias(prompt, {target_token_id: bias})
while target_token not in modified:
    bias += 5
    modified = topk_with_bias(prompt, {target_token_id: bias})

top_logprob = max(original.values())
biased_other_logprobs = np.array(
    [lp for t, lp in modified.items() if t != target_token]
)
biased_target_logprob = np.array(
    [lp for t, lp in modified.items() if t == target_token]
)
real_target_logprob = (
    biased_target_logprob - biased_other_logprobs + top_logprob - bias
)[0]

print(real_target_logprob)

assert round(real_target_logprob, 1) == -27.6